In [ ]:
pip install sentencepiece

In [ ]:
pip install bert-for-tf2

In [ ]:
import pandas as pd
import re
from sentencepiece import SentencePieceProcessor, SentencePieceTrainer
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from keras.models import load_model
import numpy as np
import pandas as pd

In [ ]:
cd drive/MyDrive/dacon_psy

[Errno 2] No such file or directory: 'drive/MyDrive/dacon_psy'
/content/drive/MyDrive/dacon_psy


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_x.csv')
y_train = train['author'].values

def cleaning_text(text):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.replace('the ','')
    text = text.replace('i ','')
    
    return text

def loading(train, test, cleaning_text):
  train_sentences = list(train['text'])
  train_texts = []
  for sen in train_sentences:
    train_texts.append(cleaning_text(sen))
  print('cleaning train texts completed')
  del train_sentences, train, sen
  test_sentences = list(test['text'])
  test_texts = []
  for sen in test_sentences:
    test_texts.append(cleaning_text(sen))
  print('cleaning test texts completed')

  with open('author.txt','w',encoding='utf-8') as f:
      f.write('\n'.join(train_texts))
  del f

  SentencePieceTrainer.Train('--input=author.txt --model_prefix=author --vocab_size=5000', max_df = 0.80, min_df = 0.25)

  sp = SentencePieceProcessor()
  sp.Load("author.model")

  cv = CountVectorizer(tokenizer = sp.encode_as_pieces)

  train_pad = cv.fit_transform(train_texts).toarray()
  del train_texts
  test_pad = cv.transform(test_texts).toarray()
  del test_texts, sp, cv
  print('data setting completed')
  del test_sentences, test, sen, cleaning_text
  return train_pad, test_pad

train_pad, test_pad = loading(train, test, cleaning_text)
print(train_pad.shape)
del SentencePieceProcessor, SentencePieceTrainer, CountVectorizer

cleaning train texts completed
cleaning test texts completed
data setting completed
(54879, 5142)


In [ ]:
def create_dnn(input_shape):
  input = layers.Input(shape=(input_shape,), name='INPUT')
  dnn = layers.Dense(1024)(input)
  batch = layers.BatchNormalization()(dnn)
  relu = layers.ReLU()(batch)
  drop = layers.Dropout(0.15)(relu)
  dnn = layers.Dense(256)(drop)
  batch = layers.BatchNormalization()(dnn)
  relu = layers.ReLU()(batch)
  drop = layers.Dropout(0.15)(relu)
  dnn = layers.Dense(64)(drop)
  batch = layers.BatchNormalization()(dnn)
  relu = layers.ReLU()(batch)
  drop = layers.Dropout(0.30)(relu)
  dnn = layers.Dense(16)(drop)
  batch = layers.BatchNormalization()(dnn)
  relu = layers.ReLU()(batch)
  drop = layers.Dropout(0.30)(relu)
  last_layer = layers.Dense(5, activation = 'softmax')(drop)

  model = models.Model(inputs=input, outputs=last_layer)
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizers.Adam(0.0005))
  return model

In [ ]:
model = create_dnn(train_pad.shape[1])
model.summary()

In [ ]:
def creating_results(seed, train_pad, test_pad, y_train, fold):
  batch_size =  32
  epochs = 300
  predicts = []
  scores = []
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
  es = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
  fold_2 = 0
  for train_index, val_index in skf.split(train_pad, y_train):
    path = 'submissions/DNN_'+str(fold)+'_'+str(fold_2)
    filepath_val_loss="models/"+path+".tf"
    checkpoint_val_loss = ModelCheckpoint(filepath_val_loss, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    model = create_dnn(train_pad.shape[1])
    tr_X = train_pad[train_index]
    tr_y = y_train[train_index]

    val_X = train_pad[val_index]
    val_y = y_train[val_index]

    model.fit(tr_X,tr_y,
                        batch_size=batch_size,
                        epochs=epochs, 
                        validation_data=(val_X,val_y), 
                        shuffle=True, 
                        verbose=0,
                        callbacks=[es,checkpoint_val_loss] #LearningRateScheduler(lr_decay)
                        ) 
    
    model = load_model(filepath_val_loss)
    predicts.append(model.predict(test_pad))
    score = log_loss(val_y,model.predict(val_X))
    scores.append(score)
    fold_2 += 1
    del tr_X,tr_y,val_X,val_y,model,score
  prediction = pd.DataFrame(np.mean(predicts,axis=0)).reset_index()
  score = np.mean(scores)
  print(score)
  file_path = 'submissions/DNN_'+str(score)[:8]+'.csv'
  prediction.to_csv(file_path, index=False) 
  print('#'*25,"FINISHED  : ",score, ' '*10,'#'*25)
  del prediction, score, file_path, filepath_val_loss,es,skf,train_index,val_index,scores,predicts,epochs,batch_size,

In [ ]:
fold = 0
for seed in [1210*x for x in range(5)]:
  fold += 1
  creating_results(seed, train_pad, test_pad, y_train, fold)

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test_x.csv')
y_train = train['author'].values

del pd

def cleaning_text(text):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.replace('the ','')
    text = text.replace('i ','')
    
    return text

import re
train_sentences = list(train['text'])
train_texts = []
for sen in train_sentences:
  train_texts.append(cleaning_text(sen))
print('cleaning train texts completed')
del train_sentences, train, sen
test_sentences = list(test['text'])
test_texts = []
for sen in test_sentences:
  test_texts.append(cleaning_text(sen))
print('cleaning test texts completed')
del test_sentences, test, sen, cleaning_text, re

cleaning train texts completed
cleaning test texts completed


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import bert

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)       

VOCAB_LENGTH = len(tokenizer.vocab)

def tokenize_text(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

train_token = [tokenize_text(text) for text in train_texts]
test_token = [tokenize_text(text) for text in test_texts]

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
texts_with_len = [[text, len(text)]
                 for i, text in enumerate(train_token)]
texts_with_len.sort(key=lambda x: x[1], reverse=True)
max_len = texts_with_len[0][1]
k = 0
for _,length in texts_with_len:
  if length == 0:
    k+= 1
texts_with_len = texts_with_len[k:]
padding_type = 'post'
train_pad = pad_sequences(train_token, padding=padding_type, maxlen=max_len)
test_pad = pad_sequences(test_token, padding=padding_type, maxlen=max_len)

In [ ]:
import numpy as np
import pandas as pd
train_df = pd.DataFrame(train_pad)
test_df = pd.DataFrame(test_pad)
print('train의 shape', train_df.shape)
print('test의 shape', test_df.shape)
print('train 중 0으로만 이루어져 있는 문장 중 max : ', np.sum(train_df == 0, axis=1).max())
print('test 중 0으로만 이루어져 있는 문장 중 max : ', np.sum(test_df == 0, axis=1).max())

print('문장이라고 인식되기 위해서는 최소 5단어 이상이라고 판단')
drop_index = train_df.loc[np.sum(train_df == 0, axis=1) > np.sum(train_df == 0, axis=1).max()-5].index
train_df = train_df.drop(index=drop_index)
y_train = pd.DataFrame(y_train).drop(index=drop_index)
print('##########너무 적은 단어 수 제거##########')
print('train 중 0으로만 이루어져 있는 문장 중 max :', np.sum(train_df == 0, axis=1).max())
print('test 중 0으로만 이루어져 있는 문장 중 max : ', np.sum(test_df == 0, axis=1).max())

train_word = train_df.applymap(lambda x : 1 if x > 0 else 0)
test_word = test_df.applymap(lambda x : 1 if x > 0 else 0)

print('\ntrain_word 중 하나의 문장으로만 이루어진 문장 시작점 : ',train_word.loc[:,(train_word.sum()==1)].columns[0])
print('test_word 중 하나의 문장으로만 이루어진 문장 시작점 : ',test_word.loc[:,(test_word.sum()==1)].columns[0])
print('train_word 중 하나의 문장으로만 이루어진 문장 끝나는 지점 : ',train_word.loc[:,(train_word.sum()==1)].columns[-1])
print('test_word 중 하나의 문장으로만 이루어진 문장 끝나는 지점 : ',test_word.loc[:,(test_word.sum()==1)].columns[-1])

print('*test의 처리를 위해서 따로 처리는 하지 않는다.*')

print('\ntrain_word 중 두개의 문장으로만 이루어진 문장 시작점 : ',train_word.loc[:,(train_word.sum()==2)].columns[0])
print('test_word 중 두개의 문장으로만 이루어진 문장 시작점 : ',test_word.loc[:,(test_word.sum()==2)].columns[0])
print('train_word 중 두개의 문장으로만 이루어진 문장 끝나는 지점 : ',train_word.loc[:,(train_word.sum()==2)].columns[-1])
print('test_word 중 두개의 문장으로만 이루어진 문장 끝나는 지점 : ',test_word.loc[:,(test_word.sum()==2)].columns[-1])

print('*test의 처리를 위해서 따로 처리는 하지 않는다.*')
print('\n최종 train shape : ',train_df.shape)
print('최종 test shape : ', test_df.shape)
print('최종 y_train shape :', y_train.shape)

train의 shape (54879, 476)
test의 shape (19617, 476)
train 중 0으로만 이루어져 있는 문장 중 max :  476
test 중 0으로만 이루어져 있는 문장 중 max :  451
문장이라고 인식되기 위해서는 최소 5단어 이상이라고 판단
##########너무 적은 단어 수 제거##########
train 중 0으로만 이루어져 있는 문장 중 max : 471
test 중 0으로만 이루어져 있는 문장 중 max :  451

train_word 중 하나의 문장으로만 이루어진 문장 시작점 :  429
test_word 중 하나의 문장으로만 이루어진 문장 시작점 :  451
train_word 중 하나의 문장으로만 이루어진 문장 끝나는 지점 :  475
test_word 중 하나의 문장으로만 이루어진 문장 끝나는 지점 :  460
*test의 처리를 위해서 따로 처리는 하지 않는다.*

train_word 중 두개의 문장으로만 이루어진 문장 시작점 :  423
test_word 중 두개의 문장으로만 이루어진 문장 시작점 :  421
train_word 중 두개의 문장으로만 이루어진 문장 끝나는 지점 :  428
test_word 중 두개의 문장으로만 이루어진 문장 끝나는 지점 :  450
*test의 처리를 위해서 따로 처리는 하지 않는다.*

최종 train shape :  (54105, 476)
최종 test shape :  (19617, 476)
최종 y_train shape : (54105, 1)


In [ ]:
train_pad = np.array(train_df)
test_pad = np.array(test_df)

In [ ]:
import numpy as np
np.save('train_pad', arr = train_pad)
np.save('test_pad', arr = test_pad)
np.save('y_train', arr=y_train)

In [ ]:
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

train_pad = np.load('train_pad.npy')
test_pad = np.load('test_pad.npy')
y_train = np.load('y_train.npy')

In [ ]:
def CNN_1234KS(VOCAB_LENGTH, EMB_DIM, input_shape):
  input = layers.Input(shape=(input_shape,), name='INPUT')
  emb = layers.Embedding(VOCAB_LENGTH, EMB_DIM, name='EMBEDDING')(input)
  first = layers.Conv1D(128, kernel_size = 1, padding='same', activation='relu', name = 'CNN_first_1')(emb)
  batch_1 = layers.BatchNormalization(name = 'CNN_first_batch')(first)
  pool_1 = layers.GlobalMaxPooling1D(name = 'CNN_first_pooling')(batch_1)
  second = layers.Conv1D(128, kernel_size = 2, padding='same', activation='relu', name = 'CNN_second_1')(emb)
  batch_2 = layers.BatchNormalization(name = 'CNN_second_batch')(second)
  pool_2 = layers.GlobalMaxPooling1D(name = 'CNN_second_pooling')(batch_2)
  third = layers.Conv1D(128, kernel_size = 3, padding='same', activation='relu', name = 'CNN_third_1')(emb)
  batch_3 = layers.BatchNormalization(name = 'CNN_third_batch')(third)
  pool_3 = layers.GlobalMaxPooling1D(name = 'CNN_third_pooling')(batch_3)
  four = layers.Conv1D(128, kernel_size = 4, padding='same', activation='relu', name = 'CNN_fourth_1')(emb)
  batch_4 = layers.BatchNormalization(name = 'CNN_fourth_batch')(four)
  pool_4 = layers.GlobalMaxPooling1D(name = 'CNN_fourth_pooling')(batch_4)
  Den_1 = layers.Dense(128, name = 'input_Dense_1')(input)
  Den_batch = layers.BatchNormalization(name = 'input_Dense_batch')(Den_1)
  concated_1D = layers.concatenate([pool_1,pool_2,pool_3, pool_4, Den_batch], name ='CONCAT')
  dropout_15 = layers.Dropout(0.15, name='FC_dropout_1')(concated_1D)
  FC = layers.Dense(128, activation = 'relu', name = 'FC_Dense_1')(dropout_15)
  dropout_15 = layers.Dropout(0.15 ,name = 'FC_dropout_2')(FC)
  FC = layers.Dense(32, activation = 'relu',name = 'FC_Dense_2')(dropout_15)
  dropout_30 = layers.Dropout(0.30, name='FC_dropout_3')(FC)
  last_layer = layers.Dense(5, activation='softmax', name='Final_Layer')(dropout_30)
  model = models.Model(inputs=input, outputs=last_layer)
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizers.Adam())
  return model

In [ ]:
def CNN_234KS(VOCAB_LENGTH, EMB_DIM, input_shape):

  input = layers.Input(shape=(input_shape,), name='INPUT')
  emb = layers.Embedding(VOCAB_LENGTH, EMB_DIM, name='EMBEDDING')(input)
  first = layers.Conv1D(128, kernel_size = 2, padding='same',  name = 'CNN_first_1')(emb)
  batch_1 = layers.BatchNormalization(name = 'CNN_first_batch')(first)
  lelu_1 = layers.LeakyReLU()(batch_1)
  pool_1 = layers.GlobalMaxPooling1D(name = 'CNN_first_pooling')(lelu_1)

  second = layers.Conv1D(128, kernel_size = 3, padding='same', name = 'CNN_second_1')(emb)
  batch_2 = layers.BatchNormalization(name = 'CNN_second_batch')(second)
  lelu_2 = layers.LeakyReLU()(batch_2)
  pool_2 = layers.GlobalMaxPooling1D(name = 'CNN_second_pooling')(batch_2)

  third = layers.Conv1D(128, kernel_size = 4, padding='same', name = 'CNN_third_1')(emb)
  batch_3 = layers.BatchNormalization(name = 'CNN_third_batch')(third)
  lelu_3 = layers.LeakyReLU()(batch_3)
  pool_3 = layers.GlobalMaxPooling1D(name = 'CNN_third_pooling')(batch_3)

  concated_1D = layers.concatenate([pool_1,pool_2,pool_3], name ='CONCAT')

  dropout_15 = layers.Dropout(0.30, name='FC_dropout_1')(concated_1D)
  FC = layers.Dense(128, name = 'FC_Dense_1')(dropout_15)
  batch = layers.BatchNormalization()(FC)
  relu = layers.ReLU()(batch)
  dropout_15 = layers.Dropout(0.15 ,name = 'FC_dropout_2')(relu)
  FC = layers.Dense(32,name = 'FC_Dense_2')(dropout_15)
  batch = layers.BatchNormalization()(FC)
  relu = layers.ReLU()(batch)
  dropout_30 = layers.Dropout(0.30, name='FC_dropout_3')(relu)

  last_layer = layers.Dense(5, activation='softmax', name='Final_Layer')(dropout_30)

  model = models.Model(inputs=input, outputs=last_layer)
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizers.Adam())
  return model

In [ ]:
def CNN_NO_OPTI(VOCAB_LENGTH, EMB_DIM, input_shape):

  input = layers.Input(shape=(input_shape,), name='INPUT')
  emb = layers.Embedding(VOCAB_LENGTH, EMB_DIM, name='EMBEDDING')(input)
  first = layers.Conv1D(128, kernel_size = 2, padding='same',  name = 'CNN_first_1')(emb)
  batch_1 = layers.BatchNormalization(name = 'CNN_first_batch')(first)
  pool_1 = layers.GlobalMaxPooling1D(name = 'CNN_first_pooling')(batch_1)

  second = layers.Conv1D(128, kernel_size = 3, padding='same', name = 'CNN_second_1')(emb)
  batch_2 = layers.BatchNormalization(name = 'CNN_second_batch')(second)
  pool_2 = layers.GlobalMaxPooling1D(name = 'CNN_second_pooling')(batch_2)

  third = layers.Conv1D(128, kernel_size = 4, padding='same', name = 'CNN_third_1')(emb)
  batch_3 = layers.BatchNormalization(name = 'CNN_third_batch')(third)
  pool_3 = layers.GlobalMaxPooling1D(name = 'CNN_third_pooling')(batch_3)

  concated_1D = layers.concatenate([pool_1,pool_2,pool_3], name ='CONCAT')

  dropout_15 = layers.Dropout(0.30, name='FC_dropout_1')(concated_1D)
  FC = layers.Dense(128, name = 'FC_Dense_1')(dropout_15)
  batch = layers.BatchNormalization()(FC)
  relu = layers.ReLU()(batch)
  dropout_15 = layers.Dropout(0.15 ,name = 'FC_dropout_2')(relu)
  FC = layers.Dense(32,name = 'FC_Dense_2')(dropout_15)
  batch = layers.BatchNormalization()(FC)
  relu = layers.ReLU()(batch)
  dropout_30 = layers.Dropout(0.30, name='FC_dropout_3')(relu)

  last_layer = layers.Dense(5, activation='softmax', name='Final_Layer')(dropout_30)

  model = models.Model(inputs=input, outputs=last_layer)
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizers.Adam())
  return model

In [ ]:
def creating_results(seed, train_pad, test_pad, y_train, fold, model, model_name):
  batch_size =  32
  epochs = 300
  predicts = []
  scores = []
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
  es = EarlyStopping(monitor='val_loss', verbose=0, patience=1)
  fold_2 = 0
  for train_index, val_index in skf.split(train_pad, y_train):
    path = 'submissions/'+model_name+'_'+str(fold)+'_'+str(fold_2)
    filepath_val_loss="models/"+path+".tf"
    checkpoint_val_loss = ModelCheckpoint(filepath_val_loss, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    tr_X = train_pad[train_index]
    tr_y = y_train[train_index]

    val_X = train_pad[val_index]
    val_y = y_train[val_index]

    model.fit(tr_X,tr_y,
                        batch_size=batch_size,
                        epochs=epochs, 
                        validation_data=(val_X,val_y), 
                        shuffle=True, 
                        verbose=0,
                        callbacks=[es,checkpoint_val_loss] #LearningRateScheduler(lr_decay)
                        ) 
    
    model = load_model(filepath_val_loss)
    predicts.append(model.predict(test_pad))
    score = log_loss(val_y,model.predict(val_X))
    scores.append(score)
    fold_2 += 1
  prediction = pd.DataFrame(np.mean(predicts,axis=0)).reset_index()
  score = np.mean(scores)
  print(score)
  file_path = 'submissions/'+model_name+'_'+str(score)[:8]+'.csv'
  prediction.to_csv(file_path, index=False) 
  print('#'*25,"FINISHED  : ",score, ' '*10,'#'*25)
  del prediction, score, file_path, filepath_val_loss,es,skf,train_index,val_index,scores,predicts,epochs,batch_size,

In [ ]:
EMB_DIM = 256
for seed in [1530*x for x in range(7)]:
  model = CNN_1234KS(VOCAB_LENGTH, EMB_DIM, train_pad.shape[1])
  model_name = 'CNN_1234KS'
  fold += 1
  creating_results(seed, train_pad, test_pad, y_train, fold, model, model_name)
for seed in [1530*x for x in range(7)]:
  model = CNN_234KS(VOCAB_LENGTH, EMB_DIM, train_pad.shape[1])
  model_name = 'CNN_234KS'
  fold += 1
  creating_results(seed, train_pad, test_pad, y_train, fold, model, model_name)
for seed in [1530*x for x in range(7)]:
  model = CNN_NO_OPTI(VOCAB_LENGTH, EMB_DIM, train_pad.shape[1])
  model_name = 'CNN_NO_OPTI'
  fold += 1
  creating_results(seed, train_pad, test_pad, y_train, fold, model, model_name)

In [ ]:
from scipy.stats import gmean
import glob
import numpy as np
import pandas as pd
from sklearn.metrics import matthews_corrcoef

In [ ]:
pred_list = glob.glob('submissions/*DNN*.csv')[:5]

sub1 = pd.read_csv(pred_list[0]).iloc[:,1:]
sub2 = pd.read_csv(pred_list[1]).iloc[:,1:]
sub3 = pd.read_csv(pred_list[2]).iloc[:,1:]
sub4 = pd.read_csv(pred_list[3]).iloc[:,1:]
sub5 = pd.read_csv(pred_list[4]).iloc[:,1:]

axis_0 = gmean([sub1.loc[:,'0'],sub2.loc[:,'0'],sub3.loc[:,'0'],sub4.loc[:,'0'],sub5.loc[:,'0']],axis=0)
axis_1 = gmean([sub1.loc[:,'1'],sub2.loc[:,'1'],sub3.loc[:,'1'],sub4.loc[:,'1'],sub5.loc[:,'1']],axis=0)
axis_2 = gmean([sub1.loc[:,'2'],sub2.loc[:,'2'],sub3.loc[:,'2'],sub4.loc[:,'2'],sub5.loc[:,'2']],axis=0)
axis_3 = gmean([sub1.loc[:,'3'],sub2.loc[:,'3'],sub3.loc[:,'3'],sub4.loc[:,'3'],sub5.loc[:,'3']],axis=0)
axis_4 = gmean([sub1.loc[:,'4'],sub2.loc[:,'4'],sub3.loc[:,'4'],sub4.loc[:,'4'],sub5.loc[:,'4']],axis=0)

DNN = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

pred_list = glob.glob('submissions/*CNN_1234KS*.csv')[:5]

sub1 = pd.read_csv(pred_list[0]).iloc[:,1:]
sub2 = pd.read_csv(pred_list[1]).iloc[:,1:]
sub3 = pd.read_csv(pred_list[2]).iloc[:,1:]
sub4 = pd.read_csv(pred_list[3]).iloc[:,1:]
sub5 = pd.read_csv(pred_list[4]).iloc[:,1:]

axis_0 = gmean([sub1.loc[:,'0'],sub2.loc[:,'0'],sub3.loc[:,'0'],sub4.loc[:,'0'],sub5.loc[:,'0']],axis=0)
axis_1 = gmean([sub1.loc[:,'1'],sub2.loc[:,'1'],sub3.loc[:,'1'],sub4.loc[:,'1'],sub5.loc[:,'1']],axis=0)
axis_2 = gmean([sub1.loc[:,'2'],sub2.loc[:,'2'],sub3.loc[:,'2'],sub4.loc[:,'2'],sub5.loc[:,'2']],axis=0)
axis_3 = gmean([sub1.loc[:,'3'],sub2.loc[:,'3'],sub3.loc[:,'3'],sub4.loc[:,'3'],sub5.loc[:,'3']],axis=0)
axis_4 = gmean([sub1.loc[:,'4'],sub2.loc[:,'4'],sub3.loc[:,'4'],sub4.loc[:,'4'],sub5.loc[:,'4']],axis=0)

CNN_1234KS = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

pred_list = glob.glob('submissions/*CNN_234KS*.csv')[:5]

sub1 = pd.read_csv(pred_list[0]).iloc[:,1:]
sub2 = pd.read_csv(pred_list[1]).iloc[:,1:]
sub3 = pd.read_csv(pred_list[2]).iloc[:,1:]
sub4 = pd.read_csv(pred_list[3]).iloc[:,1:]
sub5 = pd.read_csv(pred_list[4]).iloc[:,1:]

axis_0 = gmean([sub1.loc[:,'0'],sub2.loc[:,'0'],sub3.loc[:,'0'],sub4.loc[:,'0'],sub5.loc[:,'0']],axis=0)
axis_1 = gmean([sub1.loc[:,'1'],sub2.loc[:,'1'],sub3.loc[:,'1'],sub4.loc[:,'1'],sub5.loc[:,'1']],axis=0)
axis_2 = gmean([sub1.loc[:,'2'],sub2.loc[:,'2'],sub3.loc[:,'2'],sub4.loc[:,'2'],sub5.loc[:,'2']],axis=0)
axis_3 = gmean([sub1.loc[:,'3'],sub2.loc[:,'3'],sub3.loc[:,'3'],sub4.loc[:,'3'],sub5.loc[:,'3']],axis=0)
axis_4 = gmean([sub1.loc[:,'4'],sub2.loc[:,'4'],sub3.loc[:,'4'],sub4.loc[:,'4'],sub5.loc[:,'4']],axis=0)

CNN_234KS = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

pred_list = glob.glob('submissions/*CNN_NO_OPTI*.csv')[:5]

sub1 = pd.read_csv(pred_list[0]).iloc[:,1:]
sub2 = pd.read_csv(pred_list[1]).iloc[:,1:]
sub3 = pd.read_csv(pred_list[2]).iloc[:,1:]
sub4 = pd.read_csv(pred_list[3]).iloc[:,1:]
sub5 = pd.read_csv(pred_list[4]).iloc[:,1:]

axis_0 = gmean([sub1.loc[:,'0'],sub2.loc[:,'0'],sub3.loc[:,'0'],sub4.loc[:,'0'],sub5.loc[:,'0']],axis=0)
axis_1 = gmean([sub1.loc[:,'1'],sub2.loc[:,'1'],sub3.loc[:,'1'],sub4.loc[:,'1'],sub5.loc[:,'1']],axis=0)
axis_2 = gmean([sub1.loc[:,'2'],sub2.loc[:,'2'],sub3.loc[:,'2'],sub4.loc[:,'2'],sub5.loc[:,'2']],axis=0)
axis_3 = gmean([sub1.loc[:,'3'],sub2.loc[:,'3'],sub3.loc[:,'3'],sub4.loc[:,'3'],sub5.loc[:,'3']],axis=0)
axis_4 = gmean([sub1.loc[:,'4'],sub2.loc[:,'4'],sub3.loc[:,'4'],sub4.loc[:,'4'],sub5.loc[:,'4']],axis=0)

CNN_NO_OPTI = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

In [ ]:
axis_0 = gmean([DNN.loc[:,0],CNN_1234KS.loc[:,0]],axis=0)
axis_1 = gmean([DNN.loc[:,1],CNN_1234KS.loc[:,1]],axis=0)
axis_2 = gmean([DNN.loc[:,2],CNN_1234KS.loc[:,2]],axis=0)
axis_3 = gmean([DNN.loc[:,3],CNN_1234KS.loc[:,3]],axis=0)
axis_4 = gmean([DNN.loc[:,4],CNN_1234KS.loc[:,4]],axis=0)

first_layer = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

axis_0 = gmean([first_layer.loc[:,0],CNN_234KS.loc[:,0]],axis=0)
axis_1 = gmean([first_layer.loc[:,1],CNN_234KS.loc[:,1]],axis=0)
axis_2 = gmean([first_layer.loc[:,2],CNN_234KS.loc[:,2]],axis=0)
axis_3 = gmean([first_layer.loc[:,3],CNN_234KS.loc[:,3]],axis=0)
axis_4 = gmean([first_layer.loc[:,4],CNN_234KS.loc[:,4]],axis=0)

second_layer = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

axis_0 = gmean([second_layer.loc[:,0],CNN_NO_OPTI.loc[:,0]],axis=0)
axis_1 = gmean([second_layer.loc[:,1],CNN_NO_OPTI.loc[:,1]],axis=0)
axis_2 = gmean([second_layer.loc[:,2],CNN_NO_OPTI.loc[:,2]],axis=0)
axis_3 = gmean([second_layer.loc[:,3],CNN_NO_OPTI.loc[:,3]],axis=0)
axis_4 = gmean([second_layer.loc[:,4],CNN_NO_OPTI.loc[:,4]],axis=0)

third_layer = pd.DataFrame([axis_0,axis_1,axis_2,axis_3,axis_4]).T

In [ ]:
pred_list = [DNN, CNN_1234KS, CNN_234KS, CNN_NO_OPTI]
corrs = []
for pred in pred_list:
    corrs.append(pred.apply(lambda x : np.argmax([x[0],x[1],x[2],x[3],x[4]]),axis=1).values)
print(np.corrcoef(corrs))

[[1.         0.82534635 0.83085312 0.834004  ]
 [0.82534635 1.         0.92990904 0.93265261]
 [0.83085312 0.92990904 1.         0.95073555]
 [0.834004   0.93265261 0.95073555 1.        ]]


In [ ]:
pred_list = [first_layer, second_layer, third_layer]
corrs = []
for pred in pred_list:
    corrs.append(pred.apply(lambda x : np.argmax([x[0],x[1],x[2],x[3],x[4]]),axis=1).values)
print(np.corrcoef(corrs))

[[1.         0.95314418 0.93715232]
 [0.95314418 1.         0.97217347]
 [0.93715232 0.97217347 1.        ]]


In [ ]:
third_layer.reset_index().to_csv('final_submission_3.csv',index=False)